In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time


%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 200
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()
for i in range (1,50):
    num_classes = 2
    high = 80
    
    filter_lenghts =  [int(i) for i in np.arange(2,high,2)]
    #print(filter_lenghts)
    convs = []
    maxlen = packets
    batch_size = 1024
    epochs = 30
    numFilters=128
    activations= "sigmoid"
    dropoutVal = 0.2485206320388983
    lr = 0.001362881122433337
    adam = keras.optimizers.Adam(lr=lr)


    optim = adam
    
    input_flow = Input(shape=input_shape)

    convs= {}
    mpoolings = {}
    flattens = {}
    convs_out = []
    for i in filter_lenghts:
        convs[str(i)+'_convolution']=Conv1D(filters=numFilters,kernel_size=i,padding="valid",activation=activations,strides=1)(input_flow)

        mpoolings[str(i)+'_maxpooling'] = MaxPooling1D(pool_size= maxlen - i + 1)(convs[str(i)+'_convolution'])
        flattens[str(i)+'_flattenout'] = Flatten()(mpoolings[str(i)+'_maxpooling'])
        convs_out.append(flattens[str(i)+'_flattenout'])
    out = Concatenate()(convs_out)
    dropout = Dropout(dropoutVal)(out)
    dense = Dense(64, activation='relu')(dropout)
    dense2 = Dense(32, activation='relu')(dense)
    dropout2 = Dropout(dropoutVal)(dense2)
    end = Dense(num_classes, activation='softmax')(dropout2)

    model = Model(inputs=input_flow, outputs=end) 
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 88.64752%
loss: 0.29173%
Fit and Evaluate Time: 471.593549489975
accuracy: 89.39455%
loss: 0.25143%
Fit and Evaluate Time: 461.1194496154785
accuracy: 89.27123%
loss: 0.26567%
Fit and Evaluate Time: 461.1130793094635
accuracy: 89.68862%
loss: 0.25564%
Fit and Evaluate Time: 461.56454730033875
accuracy: 88.01907%
loss: 0.29991%
Fit and Evaluate Time: 461.1267306804657
accuracy: 89.00087%
loss: 0.28512%
Fit and Evaluate Time: 465.4665629863739
accuracy: 88.73053%
loss: 0.27735%
Fit and Evaluate Time: 460.39963817596436
accuracy: 89.24752%
loss: 0.25306%
Fit and Evaluate Time: 464.2310860157013
accuracy: 89.07914%
loss: 0.24944%
Fit and Evaluate Time: 462.9135580062866
accuracy: 85.47679%
loss: 0.32643%
Fit and Evaluate Time: 463.59921860694885
accuracy: 88.69969%
loss: 0.27395%
Fit and Evaluate Time: 462.34886145591736
accuracy: 89.11945%
loss: 0.27032%
Fit and Evaluate Time: 463.077

In [4]:
AccScores

[88.64752054214478,
 89.39455151557922,
 89.27122950553894,
 89.68861699104309,
 88.01906704902649,
 89.00087475776672,
 88.73052597045898,
 89.24751877784729,
 89.07913565635681,
 85.4767918586731,
 88.69969248771667,
 89.11945223808289,
 89.84751105308533,
 88.30127716064453,
 87.45701909065247,
 88.93447518348694,
 87.86728978157043,
 89.01748061180115,
 88.79929780960083,
 89.23328518867493,
 89.57241177558899,
 89.01036381721497,
 89.24751877784729,
 88.71866464614868,
 89.11470770835876,
 88.69495391845703,
 89.14316892623901,
 89.00087475776672,
 88.48151564598083,
 89.18111324310303,
 87.86491751670837,
 88.77795338630676,
 88.81589770317078,
 89.3068015575409,
 89.1692578792572,
 87.97875046730042,
 89.32340741157532,
 87.5329077243805,
 88.99613618850708,
 89.52972888946533,
 88.8396143913269,
 89.18823003768921,
 89.18823003768921,
 87.89100646972656,
 88.46017122268677,
 88.85858654975891,
 88.45542669296265,
 88.00009489059448,
 89.04356360435486]

In [5]:
LossScores

[0.2917253332749756,
 0.2514266508334808,
 0.2656675430544497,
 0.25564228762697877,
 0.2999102037285069,
 0.28511941850529593,
 0.27735378177696995,
 0.2530627855626656,
 0.2494403830483094,
 0.3264258473858534,
 0.2739548316494821,
 0.27031771345587713,
 0.25591995566252906,
 0.26133078872799875,
 0.2945033565054417,
 0.2548103698873168,
 0.318162184297596,
 0.25788539873409283,
 0.2702597439311003,
 0.25176019515860487,
 0.24583754484368842,
 0.2747263444159242,
 0.2613939325029828,
 0.27351009722826536,
 0.2541798573604592,
 0.29307936133473744,
 0.24245275793190313,
 0.2592257280328967,
 0.29369116100867015,
 0.2503040938319199,
 0.29898771473377994,
 0.2711869031100759,
 0.2728860194576155,
 0.24464437900555946,
 0.2596992492565525,
 0.312686061023094,
 0.2508863200366343,
 0.3381081145616061,
 0.26961057123740223,
 0.2436828354564976,
 0.25797176401899885,
 0.26381842122476673,
 0.252370147313302,
 0.3255370476134027,
 0.27916198640133794,
 0.2780020417643851,
 0.278949105995019

In [6]:
Times

[471.593549489975,
 461.1194496154785,
 461.1130793094635,
 461.56454730033875,
 461.1267306804657,
 465.4665629863739,
 460.39963817596436,
 464.2310860157013,
 462.9135580062866,
 463.59921860694885,
 462.34886145591736,
 463.07760882377625,
 460.39087891578674,
 462.88062715530396,
 462.04888916015625,
 461.48591017723083,
 463.88524746894836,
 461.8230564594269,
 463.57051944732666,
 462.1464002132416,
 463.55166697502136,
 459.5963349342346,
 461.2058870792389,
 465.3988366127014,
 462.37801456451416,
 463.9695670604706,
 459.76927876472473,
 460.57213950157166,
 462.1293878555298,
 462.8347997665405,
 462.40092515945435,
 464.17543506622314,
 462.15470814704895,
 460.14667415618896,
 461.79513716697693,
 461.5337293148041,
 462.36020946502686,
 460.29124689102173,
 463.7641336917877,
 463.566682100296,
 462.2589840888977,
 460.5436568260193,
 463.21752071380615,
 460.5419759750366,
 463.69074869155884,
 463.503347158432,
 463.32916593551636,
 462.6466703414917,
 462.6607804298401